In [14]:
import pandas as pd 
import json
import numpy as np
import torch
import re
import os
import torch.nn.functional as F
def read_question_answer_file(file_path):
    """Reads a JSONL file with question-answer data and returns a list of dictionaries."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))  # Parse each line as JSON
    return data

# Load your dataset
# dataset_path = r'med_data/phrases_no_exclude_train.jsonl' 
dataset_path = r'C:\Users\ranad\OneDrive - University of Glasgow\Attachments\Msc Final Year project\Data\MedQA-USMLE-4-options\phrases_no_exclude_train.jsonl'
questions_data = read_question_answer_file(dataset_path)


print(questions_data[0]['question'])
print(questions_data[0]['answer_idx'])
print(questions_data[0]['options'])

A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
D
{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Doxycycline', 'D': 'Nitrofurantoin'}


In [4]:
def read_json_file(file_path):
  """Reads data from a JSON file.

  Args:
    file_path: The path to the JSON file.

  Returns:
    The parsed JSON data as a Python object.
  """

  with open(file_path, 'r') as f:
    data = json.load(f)
  return data

In [26]:
import torch
import numpy as np

def calculate_entropy(logits):
  """Calculates the entropy of a probability distribution.

  Args:
    logits: A tensor of logits.

  Returns:
    The entropy value.
  """

  probs = torch.softmax(logits, dim=-1)
  log_probs = torch.log(probs + 1e-9)  # Add small epsilon for numerical stability
  entropy = -(probs * log_probs).sum(dim=-1)
  return entropy.item()

# Example usage
logits = torch.tensor([0.6863, 0.0846, 0.0907, 0.1384])
entropy = calculate_entropy(logits)
print(entropy)

1.3488813638687134


In [115]:
data = read_json_file("Logit_scores\Pubmed_FIASS\llama_RAG_CS_Logit_20240820_024752.json")
data_baseline = read_json_file("Logit_scores\Baseline\llama_RAG_CS_Logit_20240820_115302.json")
data_wiki = read_json_file("Logit_scores\Wiki_doc\llama_RAG_CS_Logit_20240821_005005.json")

In [12]:
# data
# prob_options = [F.softmax(torch.tensor(logit), dim=0).tolist() for logit in data]
# prob_options

[[0.17028051614761353,
  0.34939292073249817,
  0.24391555786132812,
  0.23641106486320496],
 [0.5550936460494995,
  0.12385813891887665,
  0.15175415575504303,
  0.16929399967193604],
 [0.30473801493644714,
  0.31196466088294983,
  0.13843348622322083,
  0.2448638677597046],
 [0.22237972915172577,
  0.18435926735401154,
  0.3312336206436157,
  0.2620273530483246],
 [0.30381619930267334,
  0.31591886281967163,
  0.09485603868961334,
  0.2854089140892029],
 [0.15729308128356934,
  0.24939759075641632,
  0.32782694697380066,
  0.2654823660850525],
 [0.1336996704339981,
  0.12268971651792526,
  0.10576514899730682,
  0.6378454566001892],
 [0.29641932249069214,
  0.31308120489120483,
  0.22028014063835144,
  0.1702193021774292],
 [0.1919691413640976,
  0.39082980155944824,
  0.26652276515960693,
  0.15067830681800842],
 [0.1692037433385849,
  0.5461964011192322,
  0.10588500648736954,
  0.178714781999588],
 [0.08590959012508392,
  0.1156039908528328,
  0.056340884417295456,
  0.74214559793

In [151]:
option_dict = {'0' : 'A', '1' : 'B', '2' : 'C', '3' : 'D' }

correct_prediction = 0
correct_positions = {}
wrong_answer_positions = []
entrophy_values = []
question_length = []
corr_pred_prob50 = 0
model_pred_50 = 0
correct_pos_with_confidence = []

def getStats(data):
    global correct_prediction, correct_positions, wrong_answer_positions, \
    entrophy_values, question_length, corr_pred_prob50, correct_pos_with_confidence,model_pred_50
    # Setting the values for different datasets
    for i , question in enumerate(questions_data[:1000]):
        max_index = torch.argmax(torch.tensor(data[i])).item()
        max_value = torch.max(F.softmax(torch.tensor(data[i]), dim=0)).item()
        # Get the corresponding option from the dictionary
        predicted_option = option_dict[str(max_index)]
        question_length.append(len(question['question'].split()))
        entrophy_values.append(calculate_entropy(torch.tensor(data[i])))
        if (predicted_option == question['answer_idx'] ):
            correct_prediction += 1
            correct_positions[str(i+1)] = F.softmax(torch.tensor(data[i]), dim=0).tolist()
        else:
            wrong_answer_positions.append(i)
        if max_value >= 0.6 :
            model_pred_50 +=1
            if (predicted_option == question['answer_idx'] ):
                corr_pred_prob50 +=1
                correct_pos_with_confidence.append(i)
                
                
# getStats(data)
# getStats(data_baseline)
getStats(data_wiki)

print("No of correct predictions: ", correct_prediction)
print("No of correct prediction with probability more than 50%: ", corr_pred_prob50)
print("No of questions with model probability more than 50% ", model_pred_50)
print("Correct questions with 80% model confidence:\n ", correct_pos_with_confidence)
print("Wrong answers:\n",wrong_answer_positions)

No of correct predictions:  432
No of correct prediction with probability more than 50%:  120
No of questions with model probability more than 50%  197
Correct questions with 80% model confidence:
  [4, 6, 14, 34, 35, 63, 77, 108, 115, 127, 131, 136, 138, 140, 145, 150, 167, 169, 174, 176, 178, 180, 183, 189, 214, 216, 219, 223, 228, 229, 231, 235, 239, 242, 245, 248, 257, 260, 263, 270, 277, 285, 286, 287, 297, 298, 312, 327, 328, 330, 349, 352, 353, 390, 395, 410, 413, 422, 426, 446, 477, 478, 481, 492, 495, 502, 504, 538, 543, 546, 550, 558, 571, 574, 582, 587, 589, 593, 598, 613, 631, 635, 643, 647, 650, 653, 663, 670, 672, 674, 686, 691, 705, 706, 724, 743, 756, 771, 776, 779, 792, 794, 813, 830, 857, 862, 882, 884, 887, 892, 900, 927, 944, 946, 947, 963, 971, 972, 980, 991]
Wrong answers:
 [7, 8, 9, 11, 15, 16, 17, 18, 19, 22, 24, 25, 26, 28, 32, 36, 38, 40, 41, 43, 46, 49, 51, 52, 54, 56, 60, 62, 64, 68, 69, 70, 71, 73, 75, 78, 79, 80, 81, 83, 84, 85, 88, 89, 91, 92, 93, 94, 98,

In [38]:
certainty = [entrophy for entrophy in entrophy_values if entrophy <1.3]

In [85]:
# questions_data[245]

In [113]:
#Long question evaluation
# long_questions = [i for i, val in enumerate(question_length) if val > 200]
# long_questions

In [114]:
# long_correct = [F.softmax(torch.tensor(data[ele]), dim=0) for ele in long_questions if str(ele) in correct_positions]
# long_correct

In [152]:
wiki_wrong = [7, 8, 9, 11, 15, 16, 17, 18, 19, 22, 24, 25, 26, 28, 32, 36, 38, 40, 41, 43, 46, 49, 51, 52, 54, 56, 60, 62, 64, 68, 69, 70, 71, 73, 75, 78, 79, 80, 81, 83, 84, 85, 88, 89, 91, 92, 93, 94, 98, 99, 100, 101, 105, 106, 107, 110, 112, 117, 118, 119, 122, 124, 125, 129, 130, 132, 133, 134, 141, 142, 144, 146, 147, 148, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 170, 179, 181, 182, 188, 192, 193, 194, 196, 197, 199, 200, 201, 203, 206, 207, 209, 210, 211, 212, 213, 215, 218, 221, 222, 224, 225, 226, 227, 230, 232, 234, 236, 237, 238, 243, 244, 246, 249, 251, 252, 254, 255, 258, 261, 264, 265, 267, 269, 271, 272, 273, 274, 276, 278, 279, 280, 281, 282, 283, 284, 290, 291, 292, 295, 300, 301, 302, 306, 308, 310, 314, 315, 316, 318, 322, 323, 325, 326, 332, 333, 334, 335, 337, 338, 339, 341, 343, 347, 348, 350, 351, 354, 355, 356, 357, 359, 360, 362, 363, 364, 366, 367, 368, 369, 370, 371, 372, 373, 375, 376, 377, 378, 379, 380, 382, 383, 385, 389, 393, 394, 398, 399, 400, 401, 402, 403, 404, 405, 407, 408, 409, 414, 420, 421, 423, 424, 425, 427, 428, 429, 430, 432, 433, 434, 439, 440, 442, 443, 444, 445, 447, 449, 450, 452, 455, 456, 457, 458, 459, 460, 461, 462, 463, 465, 466, 468, 470, 471, 472, 475, 479, 482, 483, 484, 485, 486, 488, 489, 491, 493, 494, 499, 500, 501, 505, 507, 508, 510, 511, 515, 516, 517, 518, 519, 523, 524, 528, 530, 533, 539, 540, 541, 542, 545, 547, 551, 552, 553, 554, 555, 556, 559, 560, 561, 562, 563, 565, 570, 572, 575, 576, 577, 578, 579, 580, 581, 584, 585, 586, 590, 591, 594, 596, 597, 599, 602, 603, 605, 606, 608, 610, 612, 614, 615, 616, 617, 618, 619, 622, 624, 626, 627, 628, 629, 630, 632, 633, 634, 636, 637, 638, 640, 644, 645, 646, 648, 652, 654, 656, 658, 660, 661, 662, 665, 667, 671, 673, 675, 677, 680, 681, 682, 683, 684, 685, 688, 689, 690, 692, 694, 695, 697, 698, 699, 702, 703, 704, 707, 708, 709, 710, 712, 713, 715, 716, 717, 720, 721, 722, 723, 726, 730, 733, 735, 736, 737, 738, 739, 740, 741, 744, 745, 748, 750, 751, 752, 754, 755, 758, 759, 762, 763, 764, 765, 766, 768, 769, 770, 772, 773, 774, 778, 780, 781, 782, 784, 785, 786, 790, 791, 795, 797, 798, 799, 801, 802, 803, 808, 809, 810, 811, 812, 814, 815, 817, 818, 819, 822, 823, 824, 825, 826, 828, 829, 831, 832, 838, 839, 845, 846, 847, 848, 851, 852, 853, 854, 855, 856, 859, 863, 864, 865, 866, 867, 868, 869, 871, 872, 873, 875, 876, 877, 879, 880, 881, 883, 885, 886, 888, 889, 894, 895, 896, 899, 901, 902, 903, 904, 906, 908, 909, 910, 911, 912, 915, 916, 917, 918, 920, 923, 924, 925, 928, 929, 930, 931, 932, 935, 937, 938, 939, 940, 941, 942, 943, 945, 948, 949, 950, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 964, 966, 968, 970, 973, 974, 975, 976, 977, 979, 981, 983, 984, 986, 987, 988, 990, 992, 994, 997, 999]
baseline_wrong = [1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 15, 17, 19, 21, 23, 24, 28, 30, 38, 40, 42, 45, 46, 52, 53, 57, 59, 60, 61, 64, 65, 68, 69, 70, 71, 73, 75, 76, 80, 82, 84, 85, 88, 89, 92, 93, 96, 98, 99, 100, 101, 103, 105, 106, 107, 110, 112, 117, 118, 121, 122, 125, 129, 132, 133, 134, 136, 141, 142, 143, 144, 146, 147, 148, 155, 157, 159, 160, 161, 163, 164, 167, 170, 171, 175, 178, 179, 181, 187, 192, 193, 194, 196, 197, 200, 201, 202, 204, 206, 207, 208, 209, 213, 218, 220, 222, 224, 225, 226, 227, 231, 232, 236, 237, 238, 244, 246, 247, 251, 254, 255, 261, 262, 264, 267, 269, 271, 272, 273, 274, 276, 277, 279, 281, 282, 283, 286, 288, 289, 292, 294, 295, 300, 301, 302, 304, 305, 307, 308, 310, 312, 314, 315, 316, 318, 319, 322, 323, 324, 331, 332, 334, 337, 339, 340, 343, 346, 347, 348, 351, 353, 355, 356, 357, 359, 360, 363, 364, 365, 366, 367, 369, 372, 373, 374, 375, 376, 378, 379, 382, 383, 385, 388, 389, 391, 392, 394, 401, 402, 405, 406, 407, 414, 418, 420, 421, 424, 426, 427, 428, 429, 430, 431, 432, 434, 435, 436, 438, 439, 440, 442, 444, 445, 447, 449, 450, 452, 453, 455, 456, 457, 458, 459, 461, 465, 470, 471, 472, 477, 482, 483, 484, 485, 486, 488, 489, 491, 497, 499, 501, 506, 508, 510, 514, 515, 517, 518, 519, 521, 523, 524, 527, 528, 529, 530, 531, 533, 537, 538, 546, 547, 548, 551, 552, 553, 555, 556, 561, 563, 572, 573, 575, 576, 577, 578, 579, 581, 585, 586, 588, 591, 594, 596, 597, 599, 602, 603, 605, 606, 608, 609, 611, 612, 614, 615, 616, 617, 619, 620, 624, 626, 627, 628, 629, 632, 633, 638, 642, 646, 648, 652, 655, 656, 660, 661, 662, 664, 665, 666, 667, 668, 671, 673, 677, 680, 681, 683, 685, 688, 689, 690, 692, 694, 695, 697, 698, 699, 700, 702, 704, 707, 708, 709, 710, 711, 713, 714, 715, 716, 717, 720, 721, 722, 723, 726, 728, 730, 733, 735, 736, 737, 738, 739, 741, 744, 746, 748, 749, 750, 752, 753, 754, 759, 763, 764, 765, 766, 767, 772, 773, 775, 778, 779, 780, 781, 785, 786, 790, 795, 796, 797, 799, 802, 803, 804, 805, 806, 808, 809, 811, 812, 814, 815, 816, 820, 822, 823, 824, 826, 829, 831, 832, 835, 837, 843, 845, 851, 852, 853, 856, 859, 863, 866, 867, 869, 871, 872, 873, 874, 875, 880, 881, 886, 888, 889, 890, 894, 895, 896, 898, 899, 900, 902, 908, 910, 912, 913, 914, 918, 920, 921, 923, 925, 928, 930, 932, 936, 940, 941, 942, 943, 948, 949, 951, 952, 953, 954, 955, 956, 959, 960, 961, 962, 968, 970, 973, 974, 975, 976, 977, 978, 981, 983, 984, 986, 988, 990, 991, 994, 997, 999] 
pubmed_wrong = [0, 2, 3, 4, 7, 8, 9, 10, 16, 17, 18, 19, 21, 22, 23, 24, 25, 28, 30, 37, 38, 39, 40, 41, 42, 45, 48, 52, 54, 55, 56, 57, 60, 62, 64, 68, 69, 70, 71, 73, 78, 80, 83, 84, 85, 88, 89, 92, 93, 95, 99, 100, 103, 104, 105, 107, 110, 112, 114, 116, 117, 118, 119, 120, 125, 129, 132, 133, 134, 136, 139, 141, 142, 143, 147, 148, 153, 154, 155, 156, 157, 159, 161, 162, 163, 164, 167, 168, 170, 173, 174, 179, 181, 182, 185, 188, 192, 193, 194, 196, 199, 200, 201, 202, 203, 204, 207, 209, 212, 213, 215, 217, 221, 222, 224, 225, 226, 227, 230, 231, 232, 236, 237, 241, 242, 246, 249, 251, 254, 255, 258, 261, 264, 265, 267, 269, 271, 272, 274, 276, 277, 279, 281, 283, 286, 288, 289, 290, 292, 295, 297, 298, 300, 301, 302, 304, 306, 307, 308, 310, 312, 314, 315, 316, 317, 318, 322, 323, 325, 326, 327, 332, 333, 335, 337, 339, 340, 343, 346, 347, 348, 350, 351, 354, 355, 356, 357, 359, 360, 361, 364, 365, 366, 367, 368, 369, 373, 375, 376, 377, 378, 382, 383, 386, 388, 393, 394, 396, 401, 402, 405, 406, 407, 409, 411, 414, 416, 418, 423, 424, 426, 428, 430, 432, 433, 436, 437, 438, 439, 442, 444, 445, 447, 449, 450, 452, 455, 456, 457, 458, 459, 460, 461, 462, 465, 468, 469, 470, 471, 472, 477, 479, 482, 484, 486, 488, 491, 494, 496, 497, 505, 506, 507, 508, 510, 511, 512, 513, 515, 517, 518, 519, 520, 521, 523, 524, 527, 529, 530, 531, 532, 533, 534, 536, 537, 538, 539, 541, 542, 546, 547, 548, 551, 553, 555, 556, 559, 560, 561, 562, 564, 565, 566, 572, 575, 576, 578, 579, 580, 581, 586, 590, 591, 592, 596, 599, 600, 602, 603, 605, 608, 609, 611, 612, 614, 616, 617, 618, 619, 622, 624, 626, 627, 628, 629, 633, 636, 638, 640, 642, 648, 651, 652, 654, 655, 656, 661, 662, 665, 667, 668, 671, 673, 675, 677, 680, 681, 682, 683, 685, 686, 687, 688, 689, 690, 692, 693, 694, 695, 697, 698, 699, 701, 702, 704, 707, 709, 710, 712, 713, 714, 716, 717, 720, 723, 726, 728, 730, 731, 733, 735, 736, 737, 738, 741, 744, 745, 748, 749, 750, 752, 754, 759, 760, 763, 764, 766, 768, 770, 772, 774, 778, 780, 781, 782, 785, 786, 788, 789, 790, 791, 795, 796, 798, 799, 801, 802, 803, 804, 805, 807, 808, 809, 811, 812, 815, 817, 820, 821, 822, 823, 824, 826, 831, 832, 835, 839, 840, 841, 845, 848, 851, 852, 853, 855, 856, 857, 859, 861, 862, 863, 864, 866, 867, 869, 870, 871, 873, 875, 879, 880, 884, 885, 886, 887, 888, 889, 890, 891, 894, 895, 896, 899, 900, 901, 906, 908, 910, 912, 915, 916, 918, 920, 923, 925, 928, 930, 932, 935, 937, 939, 940, 941, 942, 945, 948, 952, 953, 954, 955, 956, 958, 959, 960, 961, 963, 968, 973, 974, 975, 976, 977, 978, 983, 984, 985, 986, 987, 988, 990, 991, 992, 994, 997]

In [153]:

common_wrong_wiki_pubmed = list(set(pubmed_wrong).intersection(set(wiki_wrong)))
common_wrong_wiki_baseline = list(set(baseline_wrong).intersection(set(wiki_wrong)))
common_wrong_pubmed_baseline = list(set(pubmed_wrong).intersection(set(baseline_wrong)))

common_elements = list(set(baseline_wrong).intersection(set(wiki_wrong), set(pubmed_wrong)))




print("Common wrong ansers in wiki and pubmed:\n", common_wrong_wiki_pubmed,"\n Count: ", len(common_wrong_wiki_pubmed))
print("Common wrong ansers in wiki and baseline:\n", common_wrong_wiki_baseline,"\n Count: ", len(common_wrong_wiki_baseline)) 
print("Common wrong ansers in baseline and pubmed:\n", common_wrong_pubmed_baseline,"\n Count: ", len(common_wrong_pubmed_baseline))

print("Common wrong among all: ", common_elements, "\n Count: ", len(common_elements))

Common wrong ansers in wiki and pubmed:
 [7, 8, 9, 16, 17, 18, 19, 22, 24, 25, 28, 38, 40, 41, 52, 54, 56, 60, 62, 64, 68, 69, 70, 71, 73, 78, 80, 83, 84, 85, 88, 89, 92, 93, 99, 100, 105, 107, 110, 112, 117, 118, 119, 125, 129, 132, 133, 134, 141, 142, 147, 148, 153, 154, 155, 156, 157, 159, 161, 163, 164, 170, 179, 181, 182, 188, 192, 193, 194, 196, 199, 200, 201, 203, 207, 209, 212, 213, 215, 221, 222, 224, 225, 226, 227, 230, 232, 236, 237, 246, 249, 251, 254, 255, 258, 261, 264, 265, 267, 269, 271, 272, 274, 276, 279, 281, 283, 290, 292, 295, 300, 301, 302, 306, 308, 310, 314, 315, 316, 318, 322, 323, 325, 326, 332, 333, 335, 337, 339, 343, 347, 348, 350, 351, 354, 355, 356, 357, 359, 360, 364, 366, 367, 368, 369, 373, 375, 376, 377, 378, 382, 383, 393, 394, 401, 402, 405, 407, 409, 414, 423, 424, 428, 430, 432, 433, 439, 442, 444, 445, 447, 449, 450, 452, 455, 456, 457, 458, 459, 460, 461, 462, 465, 468, 470, 471, 472, 479, 482, 484, 486, 488, 491, 494, 505, 507, 508, 510, 511, 5

In [154]:
#Common mistakes
common_mistakes = [7, 8, 9, 17, 19, 24, 28, 38, 40, 52, 60, 64, 68, 69, 70, 71, 73, 80, 84, 85, 88, 89, 92, 93, 99, 100, 105, 107, 110, 112, 117, 118, 125, 129, 132, 133, 134, 141, 142, 147, 148, 155, 157, 159, 161, 163, 164, 170, 179, 181, 192, 193, 194, 196, 200, 201, 207, 209, 213, 222, 224, 225, 226, 227, 232, 236, 237, 246, 251, 254, 255, 261, 264, 267, 269, 271, 272, 274, 276, 279, 281, 283, 292, 295, 300, 301, 302, 308, 310, 314, 315, 316, 318, 322, 323, 332, 337, 339, 343, 347, 348, 351, 355, 356, 357, 359, 360, 364, 366, 367, 369, 373, 375, 376, 378, 382, 383, 394, 401, 402, 405, 407, 414, 424, 428, 430, 432, 439, 442, 444, 445, 447, 449, 450, 452, 455, 456, 457, 458, 459, 461, 465, 470, 471, 472, 482, 484, 486, 488, 491, 508, 510, 515, 517, 518, 519, 523, 524, 530, 533, 547, 551, 553, 555, 556, 561, 572, 575, 576, 578, 579, 581, 586, 591, 596, 599, 602, 603, 605, 608, 612, 614, 616, 617, 619, 624, 626, 627, 628, 629, 633, 638, 648, 652, 656, 661, 662, 665, 667, 671, 673, 677, 680, 681, 683, 685, 688, 689, 690, 692, 694, 695, 697, 698, 699, 702, 704, 707, 709, 710, 713, 716, 717, 720, 723, 726, 730, 733, 735, 736, 737, 738, 741, 744, 748, 750, 752, 754, 759, 763, 764, 766, 772, 778, 780, 781, 785, 786, 790, 795, 799, 802, 803, 808, 809, 811, 812, 815, 822, 823, 824, 826, 831, 832, 845, 851, 852, 853, 856, 859, 863, 866, 867, 869, 871, 873, 875, 880, 886, 888, 889, 894, 895, 896, 899, 908, 910, 912, 918, 920, 923, 925, 928, 930, 932, 940, 941, 942, 948, 952, 953, 954, 955, 956, 959, 960, 961, 968, 973, 974, 975, 976, 977, 983, 984, 986, 988, 990, 994, 997]
categories = ['Cardiovascular' ,'Respiratory','Gastrointestinal', 'Neurological','Musculoskeletal','Endocrine','Renal/Urological',
             'Dermatological','Hematological','Oncology','Reproductive / Gynecology ','Pediatrics','Infectious Diseases',
              'Geriatrics (Elderly Health)', 'Ophthalmology', 'Dentistry','General Medicine','Psychiatric and Mental Health',
              'Nutrition and Metabolism', 'Substance Abuse and Toxicology','Nutrition and Metabolism','Therapy and Rehabilitation']

In [159]:
question_category = read_json_file("Processed data/Med_classification.json")
question_category[[1,2,3,4]]

TypeError: list indices must be integers or slices, not list